## <span style="color:red">User Inputs:</span>
<span style="color:red">Filename <br>
Sample geometry</span>

In [ ]:
filename = "SQUIDdata-ppblsmo/ppblsmo05-MvsH-ZFC-100Oewarm-10K-20170124.rso.dat"
samplename = 'ppblsmo05'
width = 3. #mm
height = 5. #mm
thickness = 9. #nm

In [ ]:
#Imports, setting, etc.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy import interpolate
from scipy import optimize
import numpy.lib.recfunctions
import time

#function to read in data from a MultiVu file
def read(fname):
    #SEPARATE HEADER FROM DATA
    headerline = ''
    header = '' #header will be added to the beginning of any output files
    nline = 0
    inputfile = open(fname,'r')
    while headerline != "[Data]\n":
        headerline = inputfile.readline()
        nline += 1
        header = header + headerline
    inputfile.close()
    #READ IN DATA FILE, masked. Not all of them will be filled. The mask has values of True where data is missing
    arr = np.genfromtxt(fname, delimiter=',', skip_header=nline, names=True, usemask=True, dtype=None)
    yield arr
    yield header

Read in data in MultiVu format

In [ ]:
data, header = read(filename)
#output data columns
print header

preheader = '[Header - Processing]\n'
preheader += 'File generated at '+'[TODO:insert time here]'+' from file: '+filename+'\n'

data.dtype.names

## Checking the data to make sure it's good quality

In [ ]:
#check that we are varying the right variables over time
plt.figure()
plt.plot(data['Field_Oe'],label='Field')
plt.plot(data['Temperature_K'],label='Temp')
plt.legend()

In [ ]:
plt.figure()
plt.plot(data['Field_Oe'],data['Long_Moment_emu'])

In [ ]:
plt.figure()
plt.plot(data['Long_Reg_Fit'],'.')

Filter out curves with bad fits if necessary

In [ ]:
filterlevel = .9
filt = np.ma.where(data['Long_Reg_Fit'] < filterlevel)
#filter out values with bad fit values - this masks the whole row - useful for compressing these rows
data[filt] = np.ma.masked

Inspect raw data to see why fit failed

In [ ]:
rawfilename = filename.replace('SQUIDdata','Raw').replace('.dat','.raw')
rawdata, rawheader = read(rawfilename)

In [ ]:
size = rawdata.size/data.size
for index in filt[0]:
    plt.figure()
    plt.title('Raw Scan for H='+str(rawdata['Field_Oe'][index*size])+'Oe')
    plt.plot(rawdata['Position_cm'][index*size:(index+1)*size],rawdata['Long_Voltage'][index*size:(index+1)*size],label='Original')
    plt.plot(rawdata['Position_cm'][index*size:(index+1)*size],rawdata['Long_Average_Voltage'][index*size:(index+1)*size],label='Averaged')
    plt.plot(rawdata['Position_cm'][index*size:(index+1)*size],rawdata['Long_Reg_Fit'][index*size:(index+1)*size],label='Fit')
    plt.legend()

TODO: remove artifacts from raw data and fit, if possible

Supress masked data - since masked arrays do not work well with many numpy routines (such as interpolate)

In [ ]:
preheader += 'Data with fits below '+str(filterlevel)+' were removed from the data due to issues with the raw measurement/fit.\n'

importantcolumns = ['Temperature_K','Field_Oe','Long_Moment_emu','Long_Reg_Fit']
thedata = data[importantcolumns]
colstoextract = ~np.ma.getmaskarray(thedata).view('bool').reshape(thedata.shape + (-1,))[:,0]
gooddata = thedata[colstoextract]
gooddata

## Separate out field direction sweeps to manually determine saturation field, subtract background

In [ ]:
nextdat = gooddata[1:]
where = np.ma.where(gooddata['Field_Oe'][:-1] > nextdat['Field_Oe']) #find monotonically decreasing regions
maxfield = where[0][0]
minfield = where[0][-1] + 2 #sweeps include repeats of fields

virgin = gooddata[:maxfield] #first sweep up
sweepdown = gooddata[maxfield:minfield] #next sweep down
sweepup = gooddata[minfield:] #lastly sweep up

plt.figure()
plt.plot(virgin['Field_Oe'], virgin['Long_Moment_emu'],'b.-',label='virgin (up)')
plt.plot(sweepdown['Field_Oe'], sweepdown['Long_Moment_emu'],'r.-',label='down')
plt.plot(sweepup['Field_Oe'], sweepup['Long_Moment_emu'],'g.-',label='up')
plt.legend()

## <span style="color:red">User: Manually determine saturation region</span>

In [ ]:
uppersatfield = 3500
lowersatfield = -3500

upper = np.ma.where(gooddata['Field_Oe'] >= uppersatfield)
lower = np.ma.where(gooddata['Field_Oe'] <= lowersatfield)
plt.figure()
plt.plot(gooddata['Field_Oe'][upper],gooddata['Long_Moment_emu'][upper])
plt.plot(gooddata['Field_Oe'][lower],gooddata['Long_Moment_emu'][lower])

Fit to a linear (diamagnetic) background<br>
<span style="color:red">Option 1</span>: fit each region to a line separately

In [ ]:
lincoefflower = np.ma.polyfit(gooddata['Field_Oe'][lower], gooddata['Long_Moment_emu'][lower], 1)
lincoeffupper = np.ma.polyfit(gooddata['Field_Oe'][upper], gooddata['Long_Moment_emu'][upper], 1)
print lincoefflower
print lincoeffupper
print (lincoeffupper[0]+lincoefflower[0])/2.
print (lincoeffupper[1]+lincoefflower[1])/2.

plt.figure()
plt.plot(gooddata['Field_Oe'][upper],gooddata['Long_Moment_emu'][upper],'r.')
plt.plot(gooddata['Field_Oe'][lower],gooddata['Long_Moment_emu'][lower],'r.')
bglower = lincoefflower[0]*gooddata['Field_Oe'][lower]+lincoefflower[1]
bgupper = lincoeffupper[0]*gooddata['Field_Oe'][upper]+lincoeffupper[1]
plt.plot(gooddata['Field_Oe'][upper],bgupper,'b')
plt.plot(gooddata['Field_Oe'][lower],bglower,'b')

In [ ]:
#Option 1.1 - only use the middle region (lower) since it has the least temperature drift during the measurement
backgroundslope = lincoefflower[0]
backgroundoffset = 0

backgroundtype = 1

In [ ]:
#Option 1.2 - subtract using the average of the two sides
backgroundslope = (lincoeffupper[0]+lincoefflower[0])/2.
backgroundoffset = (lincoeffupper[1]+lincoefflower[1])/2.

backgroundtype = 2

<span style="color:red">Option 2</span>: fit both regions together to lines with the same slope and opposite intercepts

In [ ]:
#adapted from https://stackoverflow.com/questions/23532068/fitting-multiple-data-sets-using-scipy-optimize-with-the-same-parameters
init_params = [0,0]
def err1(p,x,y):
    m,b = p
    return m*x+b - y
def err2(p,x,y):
    m,b = p
    return m*x-b - y
def global_error(params, lowField, lowMoment, highField, highMoment):
    errlow = err2(params, lowField, lowMoment)
    errhigh = err1(params, highField, highMoment)
    return np.ma.concatenate([errlow,errhigh])

fit_params, success = optimize.leastsq(global_error, init_params, args=(gooddata['Field_Oe'][lower], gooddata['Long_Moment_emu'][lower], gooddata['Field_Oe'][upper], gooddata['Long_Moment_emu'][upper]))
print success
print fit_params

backgroundslope = fit_params[0]
backgroundoffset = 0

backgroundtype = 3

plt.figure()
plt.plot(gooddata['Field_Oe'][upper],gooddata['Long_Moment_emu'][upper],'r.')
plt.plot(gooddata['Field_Oe'][lower],gooddata['Long_Moment_emu'][lower],'r.')
bglower = fit_params[0]*gooddata['Field_Oe'][lower]-fit_params[1]
bgupper = fit_params[0]*gooddata['Field_Oe'][upper]+fit_params[1]
plt.plot(gooddata['Field_Oe'][upper],bgupper,'b')
plt.plot(gooddata['Field_Oe'][lower],bglower,'b')

Subtract using appropriate background subtraction method

In [ ]:
preheader += 'Background at saturation '
if backgroundtype == 1:
    preheader += '(below '+str(lowersatfield)+' Oe)'
else:
    preheader += '(above '+str(uppersatfield)+' Oe and below '+str(lowersatfield)+' Oe)'
preheader += 'was fit '
if backgroundtype == 2:
    preheader += 'separately and averaged'
elif backgroundtype == 3:
    preheader += 'together with opposite intercepts'
preheader += ' to a linear (diamagnetic) background:\n'


subtracted = gooddata['Long_Moment_emu'] - backgroundslope*gooddata['Field_Oe'] - backgroundoffset
preheader += 'Subtracted_Magnetization_emu = Long_Moment_emu - '+str(backgroundslope)+'*Field_Oe - '+str(backgroundoffset)+'\n'

volume = width/10. * height/10. * thickness/1.E7 #cm^3
moment = subtracted/volume #emu/cc
preheader += 'Moment scaled to volume assuming the following dimensions:\n'
preheader += '  width: '+str(width)+' mm, height: '+str(height)+' mm, thickness: '+str(thickness)+' nm\n'

virg = moment[:maxfield]
down = moment[maxfield:minfield]
up = moment[minfield:]
plt.figure()
plt.plot(gooddata['Field_Oe'][:maxfield], virg, 'b.-')
plt.plot(gooddata['Field_Oe'][maxfield:minfield], down,'r.-')
plt.plot(gooddata['Field_Oe'][minfield:], up, 'g.-')
plt.title('MvsH')
plt.xlabel('Applied Field H (Oe)')
plt.ylabel('Moment (emu/cc)')
ax = plt.gca()
ax.tick_params(direction='in')

## Extract parameters from MvsH curve and plot

Saturation magnetization<Br>
TODO: figure out actual saturation field rather than guessing

In [ ]:
plt.figure()
plt.plot(gooddata['Field_Oe'][upper], moment[upper],'.-')

In [ ]:
plt.figure()
plt.plot(gooddata['Field_Oe'][lower], moment[lower], '.-')

Interpolate data in order to extract zero crossings<br>
<span style="color:red">Multiple interpolation/fitting routines available</span>

In [ ]:
x = range(int(gooddata['Field_Oe'][minfield]),int(gooddata['Field_Oe'][maxfield]),1) #values to plot interpolated/fit function to verify that it's a good representation of the data

#TODO: fix interpolation and value extractions so that it is monotonic, etc.
#e.g. https://stackoverflow.com/questions/17935779/constrained-spline-fit-using-scipy-in-python
#TODO: propagate errors from saturated moment to exchange bias, remanent, coercivity, etc.
interpolationtype = 1

if interpolationtype == 1:
    #The interpolator preserves monotonicity in the interpolation data and does not overshoot if the data is not smooth. The first derivatives are guaranteed to be continuous, but the second derivatives may jump at x_k
    funcdown = interpolate.PchipInterpolator(np.ma.getdata((gooddata['Field_Oe'][maxfield:minfield])[::-1]),np.ma.getdata(down[::-1]))
    funcup = interpolate.PchipInterpolator(np.ma.getdata(gooddata['Field_Oe'][minfield:]), np.ma.getdata(up))
    yup = funcup(list(x))
    ydown = funcdown(list(x))
elif interpolationtype == 2:
    #interp1d - linear spline
    funcdown = interpolate.interp1d(gooddata['Field_Oe'][maxfield:minfield][::-1], down[::-1], kind='slinear')
    funcup = interpolate.interp1d(gooddata['Field_Oe'][minfield:], up, kind='slinear')
    yup = funcup(list(x))
    ydown = funcdown(list(x))
elif interpolationtype == 3:
    #spline with smoothing
    funcdown = interpolate.splrep((gooddata['Field_Oe'][maxfield:minfield])[::-1],down[::-1], k=3,s=12)
    funcup = interpolate.splrep(gooddata['Field_Oe'][minfield:], up,k=3,s=12)
    yup = interpolate.splev(x,funcup)
    ydown = interpolate.splev(x,funcdown)
elif interpolationtype == 4:
    #Use only for precise data, as the fitted curve passes through the given points exactly. This routine is useful for plotting a pleasingly smooth curve through a few given points for purposes of plotting.
    funcdown = interpolate.Akima1DInterpolator(np.ma.getdata(gooddata['Field_Oe'][maxfield:minfield][::-1]), np.ma.getdata(down[::-1]))
    funcup = interpolate.Akima1DInterpolator(np.ma.getdata(gooddata['Field_Oe'][minfield:]), np.ma.getdata(up))
    yup = funcup(list(x))
    ydown = funcdown(list(x))

plt.figure()
plt.plot(x,ydown,'b')
plt.plot(gooddata['Field_Oe'][maxfield:minfield],down,'r.')
plt.plot(x,yup,'g')
plt.plot(gooddata['Field_Oe'][minfield:],up,'y.')

In [ ]:
preheader += 'Curve interpolated/smoothed using '
if interpolationtype == 1:
    preheader += 'PchipInterpolator (monotonic, no smoothing)'
elif interpolationtype == 2:
    preheader += 'interp1d (linear spline)'
elif interpolationtype == 3:
    preheader += 'splrep (cubic spline with 12-point smoothing)'
elif interpolationtype == 4:
    preheader += 'Akima1DInterpolator (visually smooth curve)'
preheader += ' to extract parameters:\n'

In [ ]:
#TODO: rewrite calculations using uncertainty package, so errors propogate correctly
satmomup = moment[upper].mean()
satmomdown = moment[lower].mean()
satmoment = (satmomup-satmomdown)/2.
satmomerr = np.sqrt(moment[upper].var()+moment[lower].var())
print('Saturation moment: '+str(satmoment)+' +/- '+str(satmomerr)+' emu/cm^3')
print str(satmomup)+' (upper)'
print str(satmomdown)+' (lower)'
preheader += 'Saturation moment:    '+str(satmoment)+' +/- '+str(satmomerr)+' emu/cm^3\n'

In [ ]:
Hcoercivedown = optimize.brentq(funcdown, -100, 0)
print('Coercive field (down): '+str(Hcoercivedown)+' Oe')
Hcoerciveup = optimize.brentq(funcup, 0, 100)
print('Coercive field (up): '+str(Hcoerciveup)+' Oe')
Hcoercive = (Hcoerciveup - Hcoercivedown)/2.
print ('Coercive field: '+str(Hcoercive)+' Oe')
Hexchangebias = (Hcoerciveup + Hcoercivedown)/2.
print ('Exchange bias field: '+str(Hexchangebias)+' Oe')
preheader += 'Coercive field:       '+str(Hcoercive)+' Oe\n'
preheader += 'Exchange bias field:  '+str(Hexchangebias)+' Oe\n'

In [ ]:
remanentdown = funcdown(0)
remanentup = funcup(0)
remanent = (remanentdown - remanentup)/2.
print('Remanent Magnetization: '+str(remanent)+' emu/cc')
print remanentdown
print remanentup
print funcdown(Hexchangebias)
print funcup(Hexchangebias)
preheader += 'Remanent moment:      '+str(remanent)+' emu/cm^3\n'

In [ ]:
plt.figure()
plt.plot(gooddata['Field_Oe'][maxfield:minfield], down,'r.-',label='Sweep down')
plt.plot(gooddata['Field_Oe'][minfield:], up, 'g.-',label='Sweep up')
plt.plot(0,remanentdown,'k*',label='Remanent moment')
plt.plot(0,remanentup,'k*')
plt.plot(Hcoercivedown,0,'kx',label='Coercive field')
plt.plot(Hcoerciveup,0,'kx')
plt.hlines(satmomup,Hexchangebias,gooddata['Field_Oe'][maxfield],linestyles='dashed',label='Saturation magnetization')
plt.hlines(satmomdown,gooddata['Field_Oe'][minfield],Hexchangebias,linestyles='dashed')
plt.vlines(Hexchangebias,satmomdown,satmomup,linestyles='dotted',label='Exchange bias')
plt.title('MvsH: '+samplename)
plt.xlabel('Applied Field (Oe)')
plt.ylabel('Moment (emu/cc)')
ax = plt.gca()
plt.legend(loc=4)
ax.tick_params(direction='in')
print('Saturation moment: '+str(satmoment)+' +/- '+str(satmomerr)+' emu/cm^3')
print('Remanent moment: '+str(remanent)+' emu/cm^3')
print ('Coercive field: '+str(Hcoercive)+' Oe')
print ('Exchange bias field: '+str(Hexchangebias)+' Oe')

## Calculate expected moment
<span style="color:red">Doping, chemical composition, etc. needed</span>

In [ ]:
Ladoping = 2./3.
Mnvalence = Ladoping*3. + (1.-Ladoping)*2. -3.*2.
delectronsperuc = 5 - Mnvalence
#octahedral field splitting - 3 t2g then 2 eg bands above
if delectronsperuc <= 3:
    momperuc = delectronsperuc
elif delectronsperuc <= 6:
    momperuc = 6.-delectronsperuc
else:
    momperuc = delectronsperuc-6.
density = 6.5 #g/cm^3
numuc = density * 6.022E23 / (Ladoping*138.91 + (1.-Ladoping)*87.62 + 54.938 + 3.*15.999) #g/uc
print numuc
print momperuc
expmoment = momperuc * numuc * (2 * 9.271E-21) #2 bohr magneton per electron
print 'Expected Saturation Moment: '+str(expmoment)+' emu/cm^3'
preheader += 'Expected sat. moment: '+str(expmoment)+' emu/cm^3\n'

## Export data to external file for graphing later

In [ ]:
#columnNames = 'Field (Oe), Temperature (K), Subtracted Moment (emu/cm3), Long Reg Fit'
outfilename = filename[:-8]+'-subtracted'+filename[-8:]
outdata = np.lib.recfunctions.append_fields(gooddata, 'Subtracted_Moment_emu/cm3', data=moment)
outcols = ['Temperature_K','Field_Oe','Subtracted_Moment_emu/cm3','Long_Reg_Fit']
outcolLabels = ['Temperature (K)', 'Field (Oe)', 'Subtracted Moment (emu/cm^3)', 'Long Reg Fit']
outheader = preheader.replace('[TODO:insert time here]', time.strftime('%c',time.localtime()))+'\n'+header+','.join(outcolLabels)

np.savetxt(outfilename, outdata[outcols], delimiter=',', header=outheader)
print 'Saved to: '+outfilename